## 8. Data Analysis
###  Sample Solutions 

Import and filter the data as in the lesson notebook

In [ ]:
using DataFrames, Statistics
using CSV,Chain, DataFramesMeta, Dates
using CairoMakie

data = DataFrame(CSV.File("dc.csv",normalizenames = true))
data.Datum = @chain data.Datum begin
    # as columns of dataframes are arrays, you need to use the broadcasted `.` version of function which normally only work on scalar types
    replace.("/" => "-") 
    Date.()
end

@chain data begin
    rename!(:Temperatur_Min => :temp_min,:Temperatur_Max => :temp_max ,:Luftfeuchtigkeit => :humidity, :Niederschlag => :rain, :Niederschlagsart =>:preci_type)
    rename!(:Temperatur => :temp,:Bedeckung => :cover, :Sonnenscheindauer => :sun, :Datum => :date, :Kategorie => :weather)
end

data = @chain data begin 
    @transform(:year = year.(:date))
    @transform(:month = month.(:date))
    @transform(:day = day.(:date))
    @transform(:monthday = monthday.(:date))
end

data.monthday = string.(data.monthday)
# cut februar the 29th in every year and the year 2022 all together to make data analysis easier 
data = @subset(data,:monthday .!= "(2, 29)",:year .!= 2022)

data

### <p style='color: green'>easy</p>

#### Exercise 1 
 Create a dataframe named `millenium` only containing the values since 2000
#### Solution 1 

In [ ]:
millenium = @subset(data, :year .> 1999)

#### Exercise 2
Filter the `millenium` so that it only contains your birthdays. 
- in which year did you have the best/worst weather to celebrate (in Würzburg ;) ) ? 

#### Solution 2 

In [ ]:
my_birthday = "(5, 29)"
birthday = @subset(millenium, :monthday .== my_birthday)
sort(birthday, :sun, rev =true)
#most sunny in 2011

In [ ]:
sort(birthday, :temp, rev = true)
# coldest in 2007
# warmest in 2005

#### Exercise 3
Find out how many times since 2000 has been a "white christmas" in Würzburg (snowfall on christmas eve or the day before and temperatures below 0).
#### Solution 3 

In [ ]:
christmas_eve = @subset(millenium, :monthday .== "(12, 24)" .|| :monthday .=="(12, 23)",  :temp_max .< 0.0)

Only the years 2001 and 2007 had a white christmas in Würzburg. 

### <p style='color: orange'>medium</p>

#### Exercise 4
Write a function that checks if a year is a leapyear and returns a bool. 
#### Solution 4 

In [ ]:
function leapyear(year)
    if year % 4 == 0
        if year % 400 == 0
            return true
        elseif year % 100 == 0
            return false
        else
            return true
        end
    else 
        return false
    end
end
    

#### Exercise 5
Calculate the average temperatur per month for all years before 1973, from 1974-1999 and from 2000 onwards. Plot them into one graph. Compare these values with the values from figure `f6`. What trend can you observe? 
#### Solution 5


In [ ]:
months = split("Jan Feb Mar Apr May Jun Jul Aug Sep Okt Nov Dez")

# copy and past from f6
temp_per_month =  @chain data begin
    groupby(:month) 
    @combine(:avg_temp = mean(:temp),:avg_temp_max = mean(:temp_max),:avg_temp_min = mean(:temp_min),:avg_humi = mean(:humidity))
end

before_1973 = @chain data begin 
    @subset(:year .< 1972)
    groupby(:month)
    @combine(:avg_temp = mean(:temp))
end

after_1973 = @chain data begin 
    @subset(:year .> 1972)
    groupby(:month)
    @combine(:avg_temp = mean(:temp))
end

# set a nice looking theme
set_theme!(theme_dark())


fig_1 = Figure()
ax1 = Axis(fig_1[1,1], xticks = (1:12, months), ylabel = "Temperature[°C]")
lines!(ax1,temp_per_month.avg_temp, label = "Mean 1948-2021")
lines!(ax1,before_1973.avg_temp, label = "Mean 1948-1971")
lines!(ax1,after_1973.avg_temp, label = "Mean 1972-2021")
axislegend(ax1)
fig_1



As seen many times before, the temperature is rising. 

### <p style='color: red'>hard</p>

#### Exercise 6
Find out what the column "Bedeckung"/"cover/cloudcover" means. Plot its monthly and yearly average since the year it has been measured. Interpret the data to your best ability. 
    Hint: There are some NaN values in the dataset (even after cover has first been measured). Use the function `isnan()` to find the NaN values and remove them from the data. 
#### Solution 6

Find the first datapoint, where cover has been measured. 

In [ ]:
# dataframes can be accessed via slicing similar to 2D-arrays
first_cover = data[1,:]
for (i,cover) in enumerate(data.cover)
    if isnan(cover) == false
        first_cover = data[i,:]
        break
    end
end
first_cover
        
    

Cover has been measured since 1949. According to [wikipedia](https://de.wikipedia.org/wiki/Bew%C3%B6lkung), cover measures the amount of clouds on the sky in steps from 0-9.  

Make a subset of the data starting from 1949 with measured cover. 

In [ ]:
cover_year = @chain data begin 
    @subset(:year .>= 1949, isnan.(:cover) .== false)
    groupby(:year)
    @combine(:cover = mean(:cover))
end

cover_month =  @chain data begin 
    @subset(:year .>= 1949, isnan.(:cover) .== false)
    groupby(:month)
    @combine(:cover = mean(:cover))
end


In [ ]:
fig_2  = Figure(resolution = (1400,500))
ax2_1 = Axis(fig_2[1,1], title = "Mean monthly cover", xticks = (1:12, months), ylabel = "Cover []")
lines!(ax2_1, cover_month.cover)
ax2_2 = Axis(fig_2[1,2], title = "Mean yearly cover" ,ylabel = "Cover []")
lines!(ax2_2,collect(1949:2021), cover_year.cover)
fig_2

In the summer months, there are less cloulds in the sky than in the winter months. The yearly mean cover shows an interesting trend, as it increases with time. Maybe climate change is also responsible there?  

#### Exercise 7
Calculate the standard deviation of the yearly temperatures (column temp). Is there a significant difference between years? If so, how can we interpret this?  
#### Solution 7

In [ ]:
stdd_temp = @chain data begin 
    groupby(:year)
    @combine(:stdd_temp = std(:temp))
end

fig_3  = Figure()
ax3 = Axis(fig_3[1,1], title = "Stdd of yearly temperatures" ,ylabel = "Temperature_Diff [°C]")
lines!(ax3,collect(1948:2021), stdd_temp.stdd_temp)
fig_3

There is a huge variance between the distribution of yearly temperatures from year to year, but there is no clear change over time. 

#### Exercise 8
Split the original dataset (before the filter of the 29th of february) in two datasets using your function definied in exercise 4. One dataset should only contain leap years and the other one should only contain normal years.  
#### Solution 8

In [ ]:
# copy and paste import code from above with different names for dataframe
data_2 = DataFrame(CSV.File("dc.csv"))
data_2.Datum = @chain data_2.Datum begin
    # as columns of dataframes are arrays, you need to use the broadcasted `.` version of function which normally only work on scalar types
    replace.("/" => "-") 
    Date.()
end

@chain data_2 begin
    # "Temperatur Min" and "Temerpatur Max" have to be renamed separately as they are Strings and not Symbols due to the space  
    # never use Strings as colums names, as they always cause problems like this
    rename!(:"Temperatur Min" => :temp_min)
    rename!(:"Temperatur Max" => :temp_max)
    rename!(:Luftfeuchtigkeit => :humidity, :Niederschlag => :rain, :Niederschlagsart =>:preci_type, :Temperatur => :temp)
    rename!(:Bedeckung => :cover, :Sonnenscheindauer => :sun, :Datum => :date, :Kategorie => :weather)
end


data_2 = @chain data_2 begin 
    @transform(:year = year.(:date))
    @transform(:month = month.(:date))
    @transform(:day = day.(:date))
    @transform(:monthday = monthday.(:date))
end

data_2.monthday = string.(data_2.monthday)

data_2 = @subset(data_2,:year .!= 2022)

#use own defined function with the . operator to broadcast
leap_years = @subset(data_2, leapyear.(:year) .== true)
normal_years = @subset(data_2, leapyear.(:year) .== false);

#### Exercise 9
Compare the average temperatures per month between leap and normal years and the distribution of temps between all leap years and all non leap years. Is there a difference? 
#### Solution 9

In [ ]:
years_leap = @chain leap_years begin 
    groupby(:year)
    @combine(:temp = mean(:temp))
end

years_normal = @chain normal_years begin 
    groupby(:year)
    @combine(:temp = mean(:temp))
end

months_leap = @chain leap_years begin 
    groupby(:month)
    @combine(:temp = mean(:temp))
end

months_normal = @chain normal_years begin 
    groupby(:month)
    @combine(:temp = mean(:temp))
end


fig_4  = Figure(resolution = (1400,500))
ax4_1 = Axis(fig_4[1,1], title = "Monthly temps", xticks = (1:12, months), ylabel = "Temperature [°C]")
lines!(ax4_1,months_leap.temp,label ="Leap years")
lines!(ax4_1,months_normal.temp,label ="Normal years")
axislegend(ax4_1)

ax4_2 = Axis(fig_4[1,2], title = "Distribution of temps" ,xlabel = "Temperature [°C]")
hist!(ax4_2,years_normal.temp, normalization = :pdf,label ="Leap years", bins = 8)
hist!(ax4_2, years_leap.temp, normalization = :pdf, label ="Normal years", bins = 8)
axislegend(ax4_2)

fig_4
    

This simple analysis reveals no clear visible difference between leap and normal years in their monthly and yearly temperatures. But the sample size is quite small and the noise in the data quite high, which could mask a bias that exists, at least in [theory](https://agupubs.onlinelibrary.wiley.com/doi/pdf/10.1029/2008GL035209).   